# DNN Based Models

#### Import modules

In [1]:
import pandas as pd
import numpy as np
import os
import random
from tqdm import tqdm
import pickle
from IPython.display import Image
import seaborn as sns
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
%matplotlib inline
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
import keras.backend as K

print(tf.__version__)

2.3.1


#### Set random seeds to make your results reproducible

In [2]:
# 매번 모델링을 할 때마다 동일한 결과를 얻으려면 아래 코드를 실행해야 함.

def reset_seeds(seed, reset_graph_with_backend=None):
    if reset_graph_with_backend is not None:
        K = reset_graph_with_backend
        K.clear_session()
        tf.compat.v1.reset_default_graph()
        print("KERAS AND TENSORFLOW GRAPHS RESET")  # optional

    np.random.seed(seed)
    random.seed(seed)
    tf.compat.v1.set_random_seed(seed)
    os.environ['CUDA_VISIBLE_DEVICES'] = ''  # for GPU
    print("RANDOM SEEDS RESET")  # optional
   
reset_seeds(1)

RANDOM SEEDS RESET


### Load Data

In [3]:
df = pd.read_pickle(os.path.abspath("../dat")+'/feature_1.pkl')
test = pd.read_csv(os.path.abspath("../dat")+'/test.csv', encoding='cp949')
df

,month,item_id,item_cnt_month,category,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_4,month_item_avg_item_cnt_lag_1,month_item_avg_item_cnt_lag_2,...,cat_delta_cnt_lag,month_aut_avg_item_cnt_lag_1,month_aut_avg_item_cnt_lag_2,month_aut_avg_item_cnt_lag_3,month_aut_avg_item_cnt_lag_4,month_pub_avg_item_cnt_lag_1,month_pub_avg_item_cnt_lag_2,month_pub_avg_item_cnt_lag_3,month_pub_avg_item_cnt_lag_4,pub_date_year
83204,5,100000,1.0,잡지,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
83205,5,100004,1.0,전집,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2014
83206,5,100030,1.0,잡지,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
83207,5,100038,1.0,청소년,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2012
83208,5,100040,1.0,중고등학습서,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237802,12,203058,0.0,중고등학습서,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
237803,12,203061,0.0,IT 모바일,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
237804,12,203065,0.0,국어 외국어 사전,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
237805,12,203068,0.0,수험서 자격증,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014


### Split Data

아래와 같이 데이터를 분할:
- Train Data: 5~11월 데이터
- Test Data: 12월 데이터

In [4]:
# object => numeric
df['category'] = df['category'].astype('category').cat.codes

In [5]:
X_train = df[df.month < 12].drop(['item_cnt_month'], axis=1)
y_train = df[df.month < 12]['item_cnt_month']
X_test = df[df.month == 12].drop(['item_cnt_month'], axis=1)

In [6]:
# Categorical features
X_train_cat = np.array(X_train['category']).reshape(-1,1)
X_test_cat = np.array(X_test['category']).reshape(-1,1)

In [7]:
# Numerical features
X_train = X_train.drop(['month', 'category'], axis=1)
X_test = X_test.drop(['month', 'category'], axis=1)

In [8]:
X_train.shape, X_test.shape

((141122, 23), (13481, 23))

### Train & Evaluate the Model

Scale Data

In [9]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Random seed 변경을 통한 다수의 DNN 모델 생성

In [10]:
# 예측값을 저장할 폴더 생성
folder = '../dat/Ensemble1'
if not os.path.isdir(folder):
    os.mkdir(folder)

In [11]:
for i in tqdm(range(20)):    
    SEED = np.random.randint(1, 10000)              
    reset_seeds(SEED)
    
    # Define the NN architecture
    input_num = keras.Input(shape=(X_train.shape[1],))
    x = keras.layers.Dense(32, activation = 'elu')(input_num)
    x1 = keras.layers.Dense(32)(x)
    x1 = keras.layers.Add()([x1,x])
    
    input_cat = keras.Input(shape=(1,))
    x = keras.layers.Embedding(np.unique(X_train_cat).size, 16)(input_cat)
    x2 = keras.layers.Flatten()(x)
    
    x = keras.layers.Concatenate()([x1, x2])
    x = keras.layers.Dense(16, activation = 'elu')(x)
    x = keras.layers.Dropout(0.1)(x)
    x1 = keras.layers.Dense(16)(x)
    x = keras.layers.Add()([x1,x])
    x = keras.layers.Dense(8, activation = 'elu')(x)
    x = keras.layers.Dropout(0.1)(x)
    x1 = keras.layers.Dense(8)(x)
    x = keras.layers.Add()([x1,x])
    output = keras.layers.Dense(1, activation='relu')(x)
    model = keras.Model([input_num,input_cat], output)
    
    # Choose the optimizer and the cost function
    model.compile(loss='mse', optimizer='adam', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    
    # Train the model
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=200)]
    hist = model.fit([X_train, X_train_cat], y_train, validation_split = 0.3,
                     batch_size=2048, epochs=2000, 
                     callbacks=callbacks, shuffle=False, verbose=2)
    
    # Make submissions
    pred = model.predict([X_test, X_test_cat])
    submission = pd.DataFrame({
        "item_id": test.item_id, 
        "item_cnt_month": pred.clip(0, 20).flatten()
    })
    t = pd.Timestamp.now()
    fname = f"{folder}/loop_submission_{t.month:02}{t.day:02}_{SEED:05}.csv"
    submission.to_csv(fname, index=False)    

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

RANDOM SEEDS RESET
Epoch 1/2000
49/49 - 0s - loss: 6.8571 - rmse: 2.6186 - val_loss: 6.1618 - val_rmse: 2.4823
Epoch 2/2000
49/49 - 0s - loss: 5.1595 - rmse: 2.2715 - val_loss: 5.4430 - val_rmse: 2.3330
Epoch 3/2000
49/49 - 0s - loss: 4.8784 - rmse: 2.2087 - val_loss: 5.3265 - val_rmse: 2.3079
Epoch 4/2000
49/49 - 0s - loss: 4.7826 - rmse: 2.1869 - val_loss: 5.3672 - val_rmse: 2.3167
Epoch 5/2000
49/49 - 0s - loss: 4.6935 - rmse: 2.1665 - val_loss: 5.3174 - val_rmse: 2.3059
Epoch 6/2000
49/49 - 0s - loss: 4.6372 - rmse: 2.1534 - val_loss: 5.3054 - val_rmse: 2.3033
Epoch 7/2000
49/49 - 0s - loss: 4.6313 - rmse: 2.1520 - val_loss: 5.3190 - val_rmse: 2.3063
Epoch 8/2000
49/49 - 0s - loss: 4.5976 - rmse: 2.1442 - val_loss: 5.2723 - val_rmse: 2.2962
Epoch 9/2000
49/49 - 0s - loss: 4.6154 - rmse: 2.1483 - val_loss: 5.3107 - val_rmse: 2.3045
Epoch 10/2000
49/49 - 0s - loss: 4.5561 - rmse: 2.1345 - val_loss: 5.2670 - val_rmse: 2.2950
Epoch 11/2000
49/49 - 0s - loss: 4.5530 - rmse: 2.1338 - val

Epoch 89/2000
49/49 - 0s - loss: 4.2609 - rmse: 2.0642 - val_loss: 4.9881 - val_rmse: 2.2334
Epoch 90/2000
49/49 - 0s - loss: 4.2638 - rmse: 2.0649 - val_loss: 4.9779 - val_rmse: 2.2311
Epoch 91/2000
49/49 - 0s - loss: 4.2638 - rmse: 2.0649 - val_loss: 4.9809 - val_rmse: 2.2318
Epoch 92/2000
49/49 - 0s - loss: 4.2567 - rmse: 2.0632 - val_loss: 4.9657 - val_rmse: 2.2284
Epoch 93/2000
49/49 - 0s - loss: 4.2730 - rmse: 2.0671 - val_loss: 4.9750 - val_rmse: 2.2305
Epoch 94/2000
49/49 - 0s - loss: 4.2590 - rmse: 2.0637 - val_loss: 4.9926 - val_rmse: 2.2344
Epoch 95/2000
49/49 - 0s - loss: 4.2492 - rmse: 2.0614 - val_loss: 4.9584 - val_rmse: 2.2267
Epoch 96/2000
49/49 - 0s - loss: 4.2385 - rmse: 2.0588 - val_loss: 4.9876 - val_rmse: 2.2333
Epoch 97/2000
49/49 - 0s - loss: 4.2745 - rmse: 2.0675 - val_loss: 4.9744 - val_rmse: 2.2303
Epoch 98/2000
49/49 - 0s - loss: 4.2477 - rmse: 2.0610 - val_loss: 4.9607 - val_rmse: 2.2273
Epoch 99/2000
49/49 - 0s - loss: 4.2570 - rmse: 2.0632 - val_loss: 4.9

Epoch 177/2000
49/49 - 0s - loss: 4.1562 - rmse: 2.0387 - val_loss: 4.9855 - val_rmse: 2.2328
Epoch 178/2000
49/49 - 0s - loss: 4.1501 - rmse: 2.0372 - val_loss: 4.9592 - val_rmse: 2.2269
Epoch 179/2000
49/49 - 0s - loss: 4.1530 - rmse: 2.0379 - val_loss: 4.9664 - val_rmse: 2.2285
Epoch 180/2000
49/49 - 0s - loss: 4.1606 - rmse: 2.0397 - val_loss: 4.9601 - val_rmse: 2.2271
Epoch 181/2000
49/49 - 0s - loss: 4.1563 - rmse: 2.0387 - val_loss: 4.9732 - val_rmse: 2.2301
Epoch 182/2000
49/49 - 0s - loss: 4.1464 - rmse: 2.0363 - val_loss: 4.9708 - val_rmse: 2.2295
Epoch 183/2000
49/49 - 0s - loss: 4.1633 - rmse: 2.0404 - val_loss: 4.9555 - val_rmse: 2.2261
Epoch 184/2000
49/49 - 0s - loss: 4.1488 - rmse: 2.0369 - val_loss: 4.9684 - val_rmse: 2.2290
Epoch 185/2000
49/49 - 0s - loss: 4.1627 - rmse: 2.0403 - val_loss: 4.9683 - val_rmse: 2.2290
Epoch 186/2000
49/49 - 0s - loss: 4.1524 - rmse: 2.0377 - val_loss: 4.9797 - val_rmse: 2.2315
Epoch 187/2000
49/49 - 0s - loss: 4.1286 - rmse: 2.0319 - va

49/49 - 0s - loss: 4.1011 - rmse: 2.0251 - val_loss: 4.9422 - val_rmse: 2.2231
Epoch 265/2000
49/49 - 0s - loss: 4.0825 - rmse: 2.0205 - val_loss: 4.9649 - val_rmse: 2.2282
Epoch 266/2000
49/49 - 0s - loss: 4.0776 - rmse: 2.0193 - val_loss: 4.9331 - val_rmse: 2.2211
Epoch 267/2000
49/49 - 0s - loss: 4.0962 - rmse: 2.0239 - val_loss: 4.9881 - val_rmse: 2.2334
Epoch 268/2000
49/49 - 0s - loss: 4.1076 - rmse: 2.0267 - val_loss: 4.9567 - val_rmse: 2.2264
Epoch 269/2000
49/49 - 0s - loss: 4.1034 - rmse: 2.0257 - val_loss: 4.9497 - val_rmse: 2.2248
Epoch 270/2000
49/49 - 0s - loss: 4.0713 - rmse: 2.0177 - val_loss: 4.9440 - val_rmse: 2.2235
Epoch 271/2000
49/49 - 0s - loss: 4.0903 - rmse: 2.0225 - val_loss: 4.9635 - val_rmse: 2.2279
Epoch 272/2000
49/49 - 0s - loss: 4.0859 - rmse: 2.0214 - val_loss: 4.9426 - val_rmse: 2.2232
Epoch 273/2000
49/49 - 0s - loss: 4.0733 - rmse: 2.0182 - val_loss: 4.9354 - val_rmse: 2.2216
Epoch 274/2000
49/49 - 0s - loss: 4.1022 - rmse: 2.0254 - val_loss: 4.9454 

...

Epoch 177/2000
49/49 - 0s - loss: 4.1396 - rmse: 2.0346 - val_loss: 5.0315 - val_rmse: 2.2431
Epoch 178/2000
49/49 - 0s - loss: 4.1325 - rmse: 2.0328 - val_loss: 5.0067 - val_rmse: 2.2376
Epoch 179/2000
49/49 - 0s - loss: 4.1195 - rmse: 2.0296 - val_loss: 5.0094 - val_rmse: 2.2382
Epoch 180/2000
49/49 - 0s - loss: 4.1424 - rmse: 2.0353 - val_loss: 5.0237 - val_rmse: 2.2414
Epoch 181/2000
49/49 - 0s - loss: 4.1298 - rmse: 2.0322 - val_loss: 5.0281 - val_rmse: 2.2424
Epoch 182/2000
49/49 - 0s - loss: 4.1313 - rmse: 2.0326 - val_loss: 5.0231 - val_rmse: 2.2412
Epoch 183/2000
49/49 - 0s - loss: 4.1161 - rmse: 2.0288 - val_loss: 5.0239 - val_rmse: 2.2414
Epoch 184/2000
49/49 - 0s - loss: 4.1220 - rmse: 2.0303 - val_loss: 5.0200 - val_rmse: 2.2405
Epoch 185/2000
49/49 - 0s - loss: 4.1210 - rmse: 2.0300 - val_loss: 5.0385 - val_rmse: 2.2447
Epoch 186/2000
49/49 - 0s - loss: 4.1341 - rmse: 2.0333 - val_loss: 5.0161 - val_rmse: 2.2397
Epoch 187/2000
49/49 - 0s - loss: 4.1319 - rmse: 2.0327 - va

49/49 - 0s - loss: 4.0680 - rmse: 2.0169 - val_loss: 5.0161 - val_rmse: 2.2397
Epoch 265/2000
49/49 - 0s - loss: 4.0656 - rmse: 2.0163 - val_loss: 5.0061 - val_rmse: 2.2374
Epoch 266/2000
49/49 - 0s - loss: 4.0698 - rmse: 2.0174 - val_loss: 5.0010 - val_rmse: 2.2363
Epoch 267/2000
49/49 - 0s - loss: 4.0627 - rmse: 2.0156 - val_loss: 5.0031 - val_rmse: 2.2368
Epoch 268/2000
49/49 - 0s - loss: 4.0662 - rmse: 2.0165 - val_loss: 5.0269 - val_rmse: 2.2421
Epoch 269/2000
49/49 - 0s - loss: 4.0539 - rmse: 2.0134 - val_loss: 5.0012 - val_rmse: 2.2363
Epoch 270/2000
49/49 - 0s - loss: 4.0397 - rmse: 2.0099 - val_loss: 4.9887 - val_rmse: 2.2335
Epoch 271/2000
49/49 - 0s - loss: 4.0773 - rmse: 2.0192 - val_loss: 5.0086 - val_rmse: 2.2380
Epoch 272/2000
49/49 - 0s - loss: 4.0671 - rmse: 2.0167 - val_loss: 5.0004 - val_rmse: 2.2362
Epoch 273/2000
49/49 - 0s - loss: 4.0551 - rmse: 2.0137 - val_loss: 4.9985 - val_rmse: 2.2357
Epoch 274/2000
49/49 - 0s - loss: 4.0352 - rmse: 2.0088 - val_loss: 4.9935 

Epoch 352/2000
49/49 - 0s - loss: 4.0057 - rmse: 2.0014 - val_loss: 5.0329 - val_rmse: 2.2434
Epoch 353/2000
49/49 - 0s - loss: 4.0044 - rmse: 2.0011 - val_loss: 4.9937 - val_rmse: 2.2347
Epoch 354/2000
49/49 - 0s - loss: 4.0033 - rmse: 2.0008 - val_loss: 5.0422 - val_rmse: 2.2455
Epoch 355/2000
49/49 - 0s - loss: 4.0046 - rmse: 2.0011 - val_loss: 5.0047 - val_rmse: 2.2371
Epoch 356/2000
49/49 - 0s - loss: 4.0008 - rmse: 2.0002 - val_loss: 5.0206 - val_rmse: 2.2407
Epoch 357/2000
49/49 - 0s - loss: 4.0015 - rmse: 2.0004 - val_loss: 5.0095 - val_rmse: 2.2382
Epoch 358/2000
49/49 - 0s - loss: 3.9880 - rmse: 1.9970 - val_loss: 4.9844 - val_rmse: 2.2326
Epoch 359/2000
49/49 - 0s - loss: 4.0117 - rmse: 2.0029 - val_loss: 5.0168 - val_rmse: 2.2398
Epoch 360/2000
49/49 - 0s - loss: 3.9948 - rmse: 1.9987 - val_loss: 5.0180 - val_rmse: 2.2401
Epoch 361/2000
49/49 - 0s - loss: 3.9999 - rmse: 2.0000 - val_loss: 5.0267 - val_rmse: 2.2420
Epoch 362/2000
49/49 - 0s - loss: 3.9975 - rmse: 1.9994 - va

49/49 - 0s - loss: 3.9583 - rmse: 1.9895 - val_loss: 5.0081 - val_rmse: 2.2379
Epoch 440/2000
49/49 - 0s - loss: 3.9564 - rmse: 1.9891 - val_loss: 5.0055 - val_rmse: 2.2373
Epoch 441/2000
49/49 - 0s - loss: 3.9573 - rmse: 1.9893 - val_loss: 5.0327 - val_rmse: 2.2434
Epoch 442/2000
49/49 - 0s - loss: 3.9494 - rmse: 1.9873 - val_loss: 5.0335 - val_rmse: 2.2435
Epoch 443/2000
49/49 - 0s - loss: 3.9447 - rmse: 1.9861 - val_loss: 5.0108 - val_rmse: 2.2385
Epoch 444/2000
49/49 - 0s - loss: 3.9588 - rmse: 1.9897 - val_loss: 5.0117 - val_rmse: 2.2387
Epoch 445/2000
49/49 - 0s - loss: 3.9469 - rmse: 1.9867 - val_loss: 5.0196 - val_rmse: 2.2404
Epoch 446/2000
49/49 - 0s - loss: 3.9567 - rmse: 1.9891 - val_loss: 5.0322 - val_rmse: 2.2433
Epoch 447/2000
49/49 - 0s - loss: 3.9551 - rmse: 1.9888 - val_loss: 5.0349 - val_rmse: 2.2438
Epoch 448/2000
49/49 - 0s - loss: 3.9340 - rmse: 1.9834 - val_loss: 5.0210 - val_rmse: 2.2408
Epoch 449/2000
49/49 - 0s - loss: 3.9410 - rmse: 1.9852 - val_loss: 5.0235 

100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [33:40<00:00, 101.02s/it]


# End